In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ln -s /content/drive/My\ Drive/ /mydrive
!ls /mydrive/YoloV4/
!unzip "/mydrive/YoloV4/train/dataset2.zip" -d /mydrive/YoloV4/train


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15873, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 15873 (delta 12), reused 7 (delta 7), pack-reused 15850 (from 3)
Receiving objects: 100% (15873/15873), 14.50 MiB | 8.72 MiB/s, done.
Resolving deltas: 100% (10679/10679), done.


# 修改配置


In [8]:
file_path = "darknet/Makefile"

with open(file_path, "r") as file:
    content = file.read()

content = content.replace("GPU=0", "GPU=1")
content = content.replace("OPENCV=0", "OPENCV=1")
content = content.replace("CUDNN=0", "CUDNN=1")
content = content.replace("CUDNN_HALF=0", "CUDNN_HALF=1")

with open(file_path, "w") as file:
    file.write(content)

print("Makefile 修改完成！")


Makefile 修改完成！


In [9]:
!cat darknet/Makefile | grep -E "GPU=|OPENCV=|CUDNN=|CUDNN_HALF="

GPU=1
CUDNN=1
CUDNN_HALF=1
OPENCV=1
# set GPU=1 and CUDNN=1 to speedup on GPU
# set CUDNN_HALF=1 to further speedup 3 x times (Mixed-precision on Tensor Cores) GPU: Volta, Xavier, Turing, Ampere, Ada and higher


# 編譯


In [10]:
!cd darknet && make

mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: wa

# Sample產生

In [ ]:
# 確保 weights 資料夾已經存在
!mkdir -p darknet/weights

# 使用 wget 下載 yolov4-tiny.weights
!wget -P darknet/weights https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

import os

# 切換到 darknet 目錄
os.chdir('/content/darknet')

# 執行 YOLO 檢測
!./darknet detector test cfg/coco.data cfg/yolov4-tiny.cfg weights/yolov4-tiny.weights data/person.jpg


# 訓練


### 準備訓練資料


In [ ]:
import os
from PIL import Image

# 設定原始標註文件路徑
input_file = "/content/drive/MyDrive/YoloV4/train/_annotations.txt"  # 替換為您的文件名
output_train_file = "/content/drive/MyDrive/YoloV4/train.txt"
output_label_dir = "/content/drive/MyDrive/YoloV4/train"  # YOLO 標註文件輸出目錄

# 創建輸出標註目錄
os.makedirs(output_label_dir, exist_ok=True)

# 解析標註並生成 YOLO 格式文件
with open(input_file, "r") as infile, open(output_train_file, "w") as outfile:
    for line in infile:
        parts = line.strip().split()  # 按空格分割
        image_path = parts[0]  # 圖片完整路徑
        bbox_info = parts[1:]  # 邊界框資訊

        # 確保圖片存在
        full_image_path = os.path.join(os.path.dirname(input_file), image_path)
        if not os.path.exists(full_image_path):
            print(f"圖片不存在: {full_image_path}")
            continue

        # 獲取圖片大小
        with Image.open(full_image_path) as img:
            img_width, img_height = img.size

        # 生成對應的 YOLO 標註文件
        label_file = os.path.join(output_label_dir, os.path.splitext(os.path.basename(image_path))[0] + ".txt")
        with open(label_file, "w") as label_outfile:
            for bbox in bbox_info:
                try:
                    bbox_parts = bbox.split(",")
                    x_min, y_min, x_max, y_max, class_id = map(int, bbox_parts)

                    # 計算 YOLO 格式 (x_center, y_center, width, height)
                    x_center = (x_min + x_max) / 2.0
                    y_center = (y_min + y_max) / 2.0
                    width = x_max - x_min
                    height = y_max - y_min

                    # 將座標轉換為相對比例
                    x_center /= img_width
                    y_center /= img_height
                    width /= img_width
                    height /= img_height

                    # 檢查座標是否有效
                    if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and width > 0 and height > 0):
                        print(f"無效的標註座標: {bbox} 在 {image_path}")
                        continue

                    # 保存 YOLO 格式
                    label_outfile.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
                except ValueError:
                    print(f"無法解析標註: {bbox} 在 {image_path}")

        # 將圖片路徑寫入 train.txt
        outfile.write(f"{os.path.join(output_label_dir, image_path)}\n")

print(f"已生成 YOLO 標註文件至 {output_label_dir}")
print(f"已生成 {output_train_file}")


已生成 YOLO 標註文件至 /content/drive/MyDrive/YoloV4/train
已生成 /content/drive/MyDrive/YoloV4/train.txt


In [11]:
# prompt: 找到/mydrive/YoloV4/train/裡面所有.jpg 匯出所有的名稱train.txt 給yolo使用

import os

# 設定圖片目錄
image_dir = "/mydrive/YoloV4/train"

# 建立 train.txt 文件
with open("/mydrive/YoloV4/train2.txt", "w") as train_file:
    # 遍歷圖片目錄
    for filename in os.listdir(image_dir):
        # 檢查檔案是否為 .jpg 檔案
        if filename.endswith(".jpg"):
            # 將圖片完整路徑寫入 train.txt
            train_file.write(os.path.join(image_dir, filename) + "\n")

print("已生成 train2.txt 文件。")

已生成 train2.txt 文件。


### 訓練


In [12]:
# 執行 YOLO 檢測
!/content/darknet/darknet detector train /content/drive/MyDrive/YoloV4/coco.data /content/drive/MyDrive/YoloV4/yolov4-tiny-obj.cfg /content/drive/MyDrive/YoloV4/yolov4-tiny.weights -dont_show


串流輸出內容已截斷至最後 5000 行。
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 14 Avg (IOU: 0.853852), count: 7, class_loss = 0.625841, iou_loss = 2.432543, total_loss = 3.058384 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 21 Avg (IOU: 0.704372), count: 4, class_loss = 0.590211, iou_loss = 8.530247, total_loss = 9.120458 
 total_bbox = 396755, rewritten_bbox = 0.964071 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 14 Avg (IOU: 0.869894), count: 2, class_loss = 0.000757, iou_loss = 0.628555, total_loss = 0.629312 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 21 Avg (IOU: 0.863148), count: 2, class_loss = 0.019110, iou_loss = 2.267149, total_loss = 2.286258 
 total_bbox = 396759, rewritten_bbox = 0.964061 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 14 Avg (IOU: 0.787499), count: 5, class_loss = 0.482155, iou_loss = 0.982107, total_loss = 1.464261 
v3 (iou loss, Normalizer: (iou: 0.07,

# 開始測試


In [ ]:
import os
import glob

# 切換到 darknet 目錄
os.chdir('/content/darknet')

# 定義資料夾路徑
train_folder = '/content/drive/MyDrive/YoloV4/train'
output_folder = '/content/drive/MyDrive/YoloV4/predicted_images'

# 確保輸出資料夾存在
os.makedirs(output_folder, exist_ok=True)

# 獲取所有圖片檔案
image_files = glob.glob(os.path.join(train_folder, '*'))

# 遍歷所有圖片
for image_file in image_files:
    # 執行 YOLO 檢測
    os.system(f"./darknet detector test /content/drive/MyDrive/YoloV4/coco.data \
        /content/drive/MyDrive/YoloV4/yolov4-tiny-obj.cfg \
        /content/drive/MyDrive/YoloV4/yolov4-tiny-obj_final.weights {image_file}")

    # 提取圖片檔名
    base_name = os.path.basename(image_file)
    predicted_name = os.path.splitext(base_name)[0] + "_prediction.jpg"

    # 移動預測圖片到指定目錄
    os.system(f"mv /content/darknet/predictions.jpg {os.path.join(output_folder, predicted_name)}")


In [13]:
import os
os.chdir('/content/darknet')
!./darknet detector demo /content/drive/MyDrive/YoloV4/coco.data \
        /content/drive/MyDrive/YoloV4/yolov4-tiny-obj.cfg \
        /content/drive/MyDrive/YoloV4/backup3/yolov4-tiny-obj_final.weights \
        /content/drive/MyDrive/YoloV4/影片/youtube_2k9b98HXWlk_1920x1080_h264.mp4 \
        -out_filename /content/drive/MyDrive/YoloV4/影片/youtube_2k9b98HXWlk_predictions2.mp4


串流輸出內容已截斷至最後 5000 行。

 cvWriteFrame 
Objects:

car: 37% 

FPS:14.4 	 AVG_FPS:19.5
OpenCV exception: show_image_mat 

 cvWriteFrame 
Objects:

car: 40% 

FPS:14.6 	 AVG_FPS:19.5
OpenCV exception: show_image_mat 

 cvWriteFrame 
Objects:

car: 41% 

FPS:14.6 	 AVG_FPS:19.5
OpenCV exception: show_image_mat 

 cvWriteFrame 
Objects:

car: 82% 

FPS:14.1 	 AVG_FPS:19.5
OpenCV exception: show_image_mat 

 cvWriteFrame 
Objects:

car: 44% 

FPS:13.9 	 AVG_FPS:19.5
OpenCV exception: show_image_mat 

 cvWriteFrame 
Objects:

car: 47% 

FPS:13.4 	 AVG_FPS:19.5
OpenCV exception: show_image_mat 

 cvWriteFrame 
Objects:

car: 32% 

FPS:13.2 	 AVG_FPS:19.5
OpenCV exception: show_image_mat 

 cvWriteFrame 
Objects:


FPS:12.9 	 AVG_FPS:19.5
OpenCV exception: show_image_mat 

 cvWriteFrame 
Objects:

car: 28% 

FPS:13.2 	 AVG_FPS:19.5
OpenCV exception: show_image_mat 

 cvWriteFrame 
Objects:

car: 34% 

FPS:12.8 	 AVG_FPS:15.2
OpenCV exception: show_image_mat 

 cvWriteFrame 
Objects:

car: 48% 

FP